In [1]:
import glob
from dalab import read_pickle
import pandas as pd
import numpy as np
from collections import Counter
from nltk import word_tokenize

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import spacy
from time import time
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, SimpleRNN, GRU, LSTM
from keras.layers import Input, Dense
from keras.models import Model
from keras.models import Sequential
from matplotlib import pyplot as plt

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = read_pickle('data/20_newsgroup/dataframes/raw_news.pickle')
df = df.sample(frac=1)
df = df.drop_duplicates(subset='text')
df.head()

,label,text
18581,forsale,Xref: cantaloupe.srv.cs.cmu.edu comp.sys.mac.w...
12950,hardware,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...
10545,motorcycles,Newsgroups: rec.motorcycles\nPath: cantaloupe....
11652,x,Newsgroups: comp.windows.x\nPath: cantaloupe.s...
6833,hockey,Newsgroups: rec.sport.hockey\nPath: cantaloupe...


In [3]:
df.iloc[1].text

'Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!agate!dog.ee.lbl.gov!hellgate.utah.edu!cc.usu.edu!slzw0\nFrom: slzw0@cc.usu.edu\nNewsgroups: comp.sys.ibm.pc.hardware\nSubject: ***Wanted : 386DX-33 motherboard\nMessage-ID: <1993Apr22.223127.66661@cc.usu.edu>\nDate: 22 Apr 93 22:31:27 MDT\nOrganization: Utah State University\nLines: 10\n\n\n  Are there anyone who wants to sell used 386dx-33 motherboard?\nIf you have one please let me know the price and the specification\n\nI am also interested in buying Trident VGA card  (1Meg)\n\nPark\n\n\n\n'

In [4]:
MAXLEN = 1000
VOCAB_SIZE = 20000
TRAIN_SIZE = 15000

In [5]:
nlp = spacy.load('en_core_web_sm')

RuntimeError: Language not supported: en_core_web_sm

In [ ]:
# Preprocessing here:
# Lower, remove unwanted chars, decide if is going to keep punctuations, lemmas, so on.

In [ ]:
all_words = word_tokenize(' '.join(df.text.tolist()))
word_counts = Counter(all_words).most_common(VOCAB_SIZE)
words = [w[0] for w in word_counts]

In [ ]:
embed_dic = {}
for index, word in enumerate(words):
    if index % 500 == 0: print(index)
    token = nlp(word)
    embed_dic[token.text] = token.vector

In [ ]:
embed_words = pd.DataFrame(embed_dic).T
embed_words.head(10)

In [ ]:
padding = pd.DataFrame({'<PAD>': np.zeros(shape=[1,embed_words.shape[1]])[0]}).T
embed_matrix = padding.append(embed_words)
embed_matrix.head()

In [ ]:
random_matrix = np.random.randn(embed_matrix.shape[0], embed_matrix.shape[1])
random_matrix[0] = np.zeros([1, embed_matrix.shape[1]])

In [ ]:
word_index = {j:i+1 for i,j in enumerate(embed_matrix.index.tolist()[1:])}

In [ ]:
tokenizer = Tokenizer()
tokenizer.word_index = word_index
sequences = tokenizer.texts_to_sequences(df.text)
data = pad_sequences(sequences, maxlen=MAXLEN)

In [ ]:
data = data.reshape(-1, data.shape[1], 1)
data = data/VOCAB_SIZE
data[0]

In [ ]:
onehot = pd.get_dummies(df['label'])
target_labels = onehot.columns
target = onehot.as_matrix()
target

In [ ]:
x_train = data[:TRAIN_SIZE]
x_test = data[TRAIN_SIZE:]

y_train = target[:TRAIN_SIZE]
y_test = target[TRAIN_SIZE:]

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=data.shape[1:]))
model.add(Dense(128, activation='relu'))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(35))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(target.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=128)

In [ ]:
embedding_layer = Embedding(len(embed_matrix), len(embed_matrix.columns), weights=[random_matrix],
                            input_length=data.shape[1:], trainable=False)

sequence_input = Input(shape=(MAXLEN,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Dropout(0.2)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output = Dense(target.shape[1], activation='softmax')(x)

model = Model(sequence_input, output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=128)

In [ ]:
model.summary()

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=128)